In [112]:
import json

article_urls = json.load(open('test-data/article-urls.json'))
promise_titles = json.load(open('test-data/promise-titles.json'))

promise_texts = []
for idx, title in enumerate(promise_titles):
    promise_texts.append(open('../promisedata/korean/prom%d.txt' % (idx+1)).read())

In [113]:
print(promise_titles[10])
print(promise_texts[10])

34개 상습침수 피해위험지역 침수피해 해소
1-3-1

34개상습침수피해위험지역침수피해해소

추진 상황

정상추진

사업 구분

임기 후

계속

총 사업비

888,014백만원

집 행 액

521,427백만원

 최종목표

○ 34개 상습 침수취약지역 해소로 수해로부터 안전한 도시 구현

※ 34개소 침수취약지역 중 ’16년 상반기까지 15개소 주요 대책사업 완료
    -  ’13년(5개소)  :  구의·자양,  면목동,  미아삼거리,  장위동,  가산디지털단지역  주변
    -  ’14년(4개소)  :  영림초교,  장안1동  주민센터,  선릉역,  을지로입구역  주변
    -  ’15년(3개소)  :  신촌현대백화점 일대, 상도동 성대시장 일대, 풍납동 주민센터 일대
    -  ’16년(3개소)  :  용답동답십리역  일대,  공릉초교  주변,  공항펌프장  주변

 추진계획(정책개요)

○ 침수원인 심층분석으로 지역 맞춤형 배수개선대책 체계적 추진

- 하수관 정비, 빗물펌프장 설치 등 유역단위별 종합개선대책 추진

- 저류조 설치, 도로물막이판, 유수장애교량 재설치 등 긴급대책 병행

- 침투형 빗물받이, 빗물 이용시설 등 친환경적 치수대책 함께 시행

 ’19년까지 34개 침수취약지역 해소

단위사업

’15년

’16년

’17년

∘신촌현대백화점 일대
∘성대시장 일대
∘풍납동 주민센터

∘공항펌프장 일대
∘공릉초교 일대
∘용답동답십리 일대

∘대치역사거리
∘신천지역
∘잠원지역
∘도림천(3개소)

침수 취약
지역 해소

임기후

∘강남역 일대
∘사당역 일대
∘망원유역 일대

’18년
∘광화문 일대
∘강서구청사거리
∘시흥사거리 일대
∘내방역사거리 일대
∘암사역 주변
∘오류동역 주변
∘한강로 일대
∘신월지역(2개소)
∘길동 일대

  ※ ’14년까지 침수취약지역 해소 실적(9개소)

- `13년 : 구의․자양, 면목동, 미아삼거리, 장위동, 가산디지털단지역 주변
- `14년 : 영림초교, 장안1동 주민센터, 선릉역, 을지로입구역 주변



In [115]:
import requests
import re
def download_article(url):
    article_id = re.search('aid=(\d+)', url).group(1)
    fname = 'test-data/article-%s.json' % article_id
    api_url = 'https://api.budgetwiser.org/api/news/get_by_url/'
    article_parsed = requests.get(api_url, {'url': url}).json()
    with open(fname, 'w') as f:
        json.dump(article_parsed, f)

import os.path
def load_article(url):
    article_id = re.search('aid=(\d+)', url).group(1)
    fname = 'test-data/article-%s.json' % article_id
    if not os.path.isfile(fname):
        download_article(url)
    return json.load(open(fname))

In [150]:
import logging
from konlpy.tag import Twitter, Kkma
twitter = Twitter()

kkma = Kkma()
import string
import re
remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
remove_quotes_map = dict([(ord(x), " ") for x in ".․,‘’´“”·‧<>「」【】–-()~～…"]) 
def clean_title(title):
    # Remove punctuation and quotes
    title = title.translate(remove_punct_map)
    title = title.translate(remove_quotes_map)
    # Collapse double spaces
    title = re.sub('\s+', ' ', title)
    return title

stopwords = "수 년 등 및 몇 중 네이버 뉴스".split()
stopwords += "공급 설치 조성 운영 실행 설립 확대 제공 실시 지원 검토 육성 추진 유치 강화 개선 구축 마련 확충 실시 개선 해소".split()
stopwords = set(stopwords)
pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True) if p[0] not in stopwords and p[1] in ('Noun', 'Verb', 'Alpha', 'Adjective')]
pos = lambda d: [p for p in kkma.morphs(clean_title(d)) if p not in stopwords]

In [163]:
from gensim import similarities, models, corpora

dictionary_ko = corpora.Dictionary.load('ko.dict')
index = similarities.MatrixSimilarity.load('lsi.index')
lsi = models.LsiModel.load('lsi.model')

def find_similar(tagged_text, similarity_threshold=None):
    vec_bow = dictionary_ko.doc2bow(tagged_text)
    vec_lsi = lsi[vec_bow]
    sims = enumerate(index[vec_lsi])
    if similarity_threshold:
        sims = filter(lambda item: item[1] >= similarity_threshold, sims)
    return sorted(sims, key=lambda item: -item[1])


In [164]:
import csv

article_promise_manual = {}

# Load manually tagged article-promise matches
with open('test-data/article-promise-manual.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        article_title = row[0]
        _promise_titles = [t for t in row[1:] if t]
        if article_title:
            article_promise_manual[article_title] = _promise_titles

In [181]:
#print(article_promise_manual)

similarity_threshold = 0.75

k_precision_values = []
k_recall_values = []
all_ranks = []

for url in article_urls:
    article_parsed = load_article(url)
    # title + excerpt
    n_words = 250
    text_excerpt = " ".join(list(filter(lambda word: len(word)>1, clean_title(article_parsed['text']).replace("\n", " ").split(" ")))[:n_words])
    search_text = article_parsed['title'] + " " + text_excerpt
    similar_promises = find_similar(pos(search_text))
    print("Article: %s" % article_parsed['title'])
    print(text_excerpt)
    
    manual_promises = article_promise_manual[article_parsed['title']]
    
    num_relevant = len(manual_promises)
    recommended = list(filter(lambda item: item[1] >= similarity_threshold, similar_promises))
    num_recommended = len(recommended)
    
    print("  Top-5 (of %d) algorithm matched promises:" % num_recommended)
    for rank, item in enumerate(recommended[:5]):
        print("  (Rank %d, %d%%) %s" % (rank, item[1]*100, promise_titles[item[0]]))

    print("  Manually tagged promises:")
    if not manual_promises:
        print("  (None)")

    ranks = []    
    for item in manual_promises:
        promise_idx = promise_titles.index(item)
        rank_score = (0, 0)
        for rank, sim in enumerate(similar_promises):
            if sim[0] == promise_idx:
                rank_score = (rank, sim[1])
                break
        print("  (Rank %d, %d%%) %s" % (rank_score[0], rank_score[1]*100, item))
        ranks.append(rank_score[0])
    

    if num_recommended > 0:
        max_rank = num_recommended - 1
        # Count the number of correctly recommended items by comparing ranks from prediction and ground truth
        num_relevant_recommended = len(list(filter(lambda r: r <= max_rank, ranks)))
        precision_at_k = num_relevant_recommended / num_recommended
        if num_relevant > 0:
            recall_at_k = num_relevant_recommended / num_relevant
        else:
            recall_at_k = 1  # nothing relevant, so always right
    else:
        precision_at_k = 1  # nothing recommended, so nothing wrong
        recall_at_k = 0  # nothing recommended, so nothing right
    
    print("Precision@%d = %.4f   Recall@%d = %.4f" % (num_recommended, precision_at_k, num_relevant, recall_at_k))
    k_precision_values.append(precision_at_k)
    k_recall_values.append(recall_at_k)
    all_ranks += ranks
    print()

print("Average precision: %.4f" % (sum(k_precision_values)/len(k_precision_values)))
print("Average recall: %.4f" % (sum(k_recall_values)/len(k_recall_values)))
print("Average rank: %.1f" % (sum(all_ranks)/len(all_ranks)))


Article: 국세청, 고액·상습 체납자 2만1403명 명단 공개...김우중 '369억원' 체납
서울뉴시스 서울뉴시스 서울뉴시스 서울뉴시스 고액 체납자 1위에 유지양 효자건설 회장447억원 체납유병언 세모그룹 회장 자녀 증여서 115억원 납부안해재산 추적조사 통해 1조5752억원 세금 징수193명 형사고발 세종뉴시스 박상영 기자 국세청은 11일 2억원 이상 세금을 내지 않은 고액 상습 체납자 2만1403명의 명단을 공개했다국세청은 국세 체납 이후 1년 넘게 내지 않고 버티고 있는 개인이나 법인의 명단을 매년 공개하고 있다 올해는 기준금액이 체납 3억원에서 2억원 이상으로 낮아지면서 공개인원이 작년보다 4748명 늘어났다이번에 공개된 2만1403명 개인은 1만5027명 법인은 6376곳 이었다 체납액은 11조34697억원으로 전년13조3018억원보다 8321억원 감소했다체납액 규모는 2억 5억원 구간의 인원이 1만6931명으로 전체의 792 체납액은 6조7977억원으로 전체의 593를 차지했다올해 공개된 고액 상습 체납자 1위는 유지양 효자건설 회장으로 상속세 447억원을 내지 않았다법인 중에는 건설업체 코레드하우징이 근로소득세 526억원을 납부하지 않았다유명인 중에는 김우중 대우그룹 회장이 양도소득세 369억원을 내지 않아 고액 상습 체납자 명단에 이름을 올렸다 김광진 현대스위스저축은행 회장도 증여세 239억원을 체납했다 최원석 동아그룹 회장도 양도소득세 5억7500만원을 내지 않았다유병언 세모그룹 회장 자녀 유상나 유혁기 유섬나 씨도 증여세 115억4300만원을 납부하지 않았다 연예인 중에는 구창모 씨가 양도소득세 3억8700만원을 탤런트 김혜선씨는 종합소득세 4억700만원을 납부하지 않았다국세청은 재산 추적조사 전담조직을 통해 체납자 재산 추적조사를 통해 1조5752억원의 세금을 징수하거나 조세채권을 확보했다고액체납자 중에는 위장이혼으로 재산을 숨기고 호화생활을 누리거나 타인 명의 사업장에 고미술품을 숨기는 사례가 있었다재산의 해외은닉 가능성을 최소화하기

Article: '밀양 참사'에 靑 긴박…위기센터 가동→文대통령 긴급회의
충청북도 제공 20171222뉴스1 News1 충청북도 제공 20171222뉴스1 News1 서울뉴스1 서미선 기자 26일 이른 아침 경남 밀양에 있는 요양병원인 세종병원에서 불이 나자 청와대는 오전 내내 긴박하게 움직였다이날 오전 7시32분께 세종병원 건물에서 불이 났다는 보고를 받은 청와대는 즉각 국가위기관리센터를 가동 국가안전보장회의NSC를 열었다정의용 청와대 국가안보실장은 센터에서 상황실장을 맡아 정위치에서 상황을 지휘했다문 대통령은 집무실에서 거의 단위로 화재 관련 상황보고를 받은 것으로 알려졌다문 대통령은 화재가 발생하자 직접 NSC에 내려오겠다고 말했으나 참모진이 이를 만류했다고 청와대 핵심관계자는 전했다그에 따르면 청와대 참모들은 대통령에게 저희가 상황을 현장에서 판단한 결과 대통령은 내려오는 것보다 기다려주는 낫겠다 종합적으로 화재가 진압되면 대통령에게 보고를 드리겠다고 했다문 대통령은 이같은 의견을 받아들여 국가위기관리센터에는 직접 방문하지 않고 대신 수석비서관 보좌관회의를 긴급 소집했다 회의는 오전 10시45분에 시작해 45분가량 진행됐다청와대 관계자는 경내에 있는 사람들이 회의에 모이는 것이라 소집 지시를 시작까지 시간이 오래 걸리지 않았을 것이라고 말했다민방위복을 입은 박수현 청와대 대변인은 회의 종료 10분 뒤인 오전 11시40분 춘추관에서 결과브리핑을 통해 대통령의 지시사항을 밝혔다문 대통령은 제천 화재 발생 이후 얼마 지나지 않아 밀양 세종병원에서 화재가 발생해 많은 사상자가 발생했고 현재 화재는 진압됐으나 사망자 수가 증가할 가능성도 배제할 없는 상황에 안타까움을 금할 없다고 말했다그는 Δ추가피해가 발생하지 않게 구조된 인원에 필요한 의료조치를 취해 추가 사망자 발생 최소화에 만전을 기할 Δ국무총리를 중심으로 화재 원인을 파악하고 복합건물에 대한 화재 재발방지 대책을 마련하며 이번 화재로 인한 인명 재산피해 조기 수습을 위해 범정부 차원 역량을 총집결해 지원

Article: "학부모 감당 힘들어…" 초등교사들 江南근무 손사래
도를 넘는 민원을 겪고 나니 하루라도 빨리 강남을 벗어나고 싶더라고요 교장 교감도 힘센 학부모 항의에 한마디도 하고 교사에게 무조건 죄송하다고 하라고 합니다서울 서초구의 초등학교에 근무하는 이모41 교사는 올해만 넘기면 강남 서초 학교에서 다시는 근무하지 않겠다고 말했다 지난해 2학년 학생들끼리 말다툼을 벌여 타이르고 화해시켰는데 한쪽 부모가 학교폭력위를 열라고 요구하더니 아이 큰아빠가 판사라며 소송을 걸겠다고 찾아오는 동안 시달렸다는 것이다 교사는 출퇴근이 조금 힘들어지더라도 학생 지도가 쉬운 동네로 학교를 옮기고 싶다고 말했다최근 서울에서 초등 교사 강남 공동화空洞化 현상이 벌어지고 있다 학부모 민원 등의 이유로 강남 서초구 초등학교 근무를 기피하는 중견 교사가 늘고 있기 때문이다 이들이 빠져나간 자리는 초임 교사가 메우고 있다 서울시교육청 관계자는 최근 들어 강남 서초를 나가는 교사는 늘어나는 반면 들어오려는 교사가 줄어들어 초임 교사가 강남 서초에 배치되는 비율이 늘고 있다고 말했다서울시교육청에 따르면 올해 4월 기준 강남 서초 지역 학교에 근무하는 경력 10년 미만 초등교사 비율은 서울 25개 자치구 평균35보다 높은 417였다 반면 이들 지역의 20년 이상 30년 미만 중견 교사 비율은 서울 평균18보다 낮은 135였다 강남 서초 초등교사 5명 1명은 최근 4년 신규 발령 교사다 5년 전만 해도 강남구188와 서초구170는 신규 교사 비율이 서울 평균158과 비슷한 수준이었지만 지금은 지역의 신규 교사 비율이 가장 높다교사들은 강남 근무 기피 이유로 까다로운 학부모와 학생의 높은 교육 수준 때문에 학생 지도가 어려워지고 담임 교체 요구 민원 발생이 잦아졌기 때문이라고 입을 모은다 서울시교육청 관계자도 강남 서초에서 교권 침해 사례나 민원이 발생하는 비율이 상대적으로 높다고 말했다한 초등학교 교장은 선생님이 다정하지 않다는 이유로 몇몇 학부모가 교장실로 몰려오는 일도 있었다면서 드센 

Article: 올해도 미쉐린의 입은 ‘한식’에 꽂혔다
동아일보여전한 한식의 보여줬다지난해에 이어 올해도 세계 최고 권위의 식당 평가 안내서인 미쉐린미슐랭 가이드의 선택은 한식이었다 미쉐린코리아는 8일 서울 송파구 시그니엘서울 호텔에서 열린 미쉐린 가이드 서울 2018 에서 1∼3개를 받은 식당 24곳을 발표했다 처음 발표된 지난해에 이어 번째다지난해와 마찬가지로 24곳의 식당이 미쉐린 가이드의 선택을 받았다 특히 지난해 13곳이 한식을 기반으로 하거나 한국적 식재료를 사용한 식당이었는데 올해도 13곳이 한식 바탕의 식당이다지난해에 이어 광주요그룹이 운영하는 가온과 서울신라호텔의 라연이 2년 연속 최고 등급인 3개요리가 매우 훌륭해 맛을 보기 위해 특별한 여행을 떠날 가치가 있는 식당를 받았다 세계적으로 3개를 받은 식당은 110여 곳에 불과하다 가온 김병진 총괄셰프는 지난 1년 동안 한식에 대해 고민하며 책임감을 놓치지 않으려고 노력했다 말했다가장 눈에 띄는 것은 새롭게 2개요리가 훌륭해 멀리 찾아갈 만한 가치가 있는 식당를 받은 정식당과 코지마다 정식당은 최근 미국 뉴욕에 정식Jungsik 식당을 내고 미쉐린 가이드 뉴욕에서 2개를 받았다 평가단은 김밥 비빔밥 구절판 보쌈 등에서 영감을 얻은 독특한 요리를 만들어내고 있다 평가했다별 1개요리가 훌륭한 식당를 받은 식당은 지난해보다 줄었다 대신 4개의 식당이 새로 이름을 올렸다 도사 익스퀴진 주옥은 한식을 바탕으로 요리를 내놓고 있다 미국에서 일식을 선보였던 해외 교포 백승욱 셰프가 한식을 기반으로 현대적이고 창의적인 요리를 선보인다 도사 최대한 한국적인 재료를 사용하고 재료 각각의 개성에 초점을 맞춰 새로운 한식의 맛을 창조한다 익스퀴진 신창호 셰프가 운영하는 한식 비스트로로 나물을 이용한 생선 카르파초 사골 버터 창의적인 메뉴가 돋보인다 주옥는 평가를 받았다 이유화 음식평론가는 새롭게 식당을 발굴하기보다는 틀을 유지하면서 소폭의 변화만 같다 말했다유현수 셰프가 자신의 식당을 차리면서 나간 이십사절기 1개 

Article: 지구온난화라면서 왜 이렇게 추울까...온난화의 역설
연일 계속된 한파로 28일 서울 송파구 잠실한강공원 일대 한강이 꽁꽁 얼어붙었다 연합뉴스 연일 계속된 한파로 28일 서울 송파구 잠실한강공원 일대 한강이 꽁꽁 얼어붙었다 연합뉴스 올겨울 추위는 7년 만의 강추위 28일 오전 서울 롯데월드타워 전망대에서 바라본 한강이 얼어 있다 28일 다소 누그러졌던 강추위가 29일과 30일 다시 기승을 부려 서울 아침 기온 영하 12도까지 떨어지겠다고 기상청은 예보했다뉴시스 28일 오전 서울 롯데월드타워 전망대에서 바라본 한강이 얼어 있다 28일 다소 누그러졌던 강추위가 29일과 30일 다시 기승을 부려 서울 아침 기온 영하 12도까지 떨어지겠다고 기상청은 예보했다뉴시스 그래픽기상청 자료를 바탕으로 작성 강찬수 기자 그래픽기상청 자료를 바탕으로 작성 강찬수 기자 1월 추위 북극진동 지수 영향받아 울산에 한파주의보가 나흘째 이어진 26일 동구 주전해안의 갯바위에 마치 눈이 쌓인 것처럼 바닷물이 얼어붙어 있다 연합뉴스 울산에 한파주의보가 나흘째 이어진 26일 동구 주전해안의 갯바위에 마치 눈이 쌓인 것처럼 바닷물이 얼어붙어 있다 연합뉴스 그래픽강찬수 기자 그래픽강찬수 기자 2018년 1월 북극진동 지수 1월 126일의 북극진동 지수 평균값은 03으로 나온다 2018년 1월 북극진동 지수 1월 126일의 북극진동 지수 평균값은 03으로 나온다 자료미국 국립해양대기국NOAA 자료미국 국립해양대기국NOAA 지구온난화라는데 추운가 울산에 한파주의보가 나흘째 이어진 26일 동구 주전해안길의 난간에 바닷물이 얼어붙어 고드름이 되어 있다 연합뉴스 울산에 한파주의보가 나흘째 이어진 26일 동구 주전해안길의 난간에 바닷물이 얼어붙어 고드름이 되어 있다 연합뉴스 지난 24일 시작된 올겨울 최강 한파가 일주일째 이어지고 있다서울의 최저기온은 지난 28일을 제외하고는 매일 아침 영하 10도를 밑돌았고 한낮에도 영하의 추위가 이어지고 있다혹한을 겪으면서 일부에서는 지구의 기온이 올라가는

...

Article: "어획량 0" 잡아떼던 中 어선, 참조기가 쏟아졌다
중국 어선으로 신속 접근 지난 15일 전남 가거도에서 서쪽으로 70여㎞ 떨어진 바다에서 서해지방해경청 3015 경비함 소속 고속단정이 중국 어선으로 접근하고 있다 사이렌을 울리며 내달린 해경 특수기동대원들은 어선에 올라 선장과 선원들을 통제하고 조업 규정 위반을 적발했다 성형주 기자 중국 어선으로 신속 접근 지난 15일 전남 가거도에서 서쪽으로 70여㎞ 떨어진 바다에서 서해지방해경청 3015 경비함 소속 고속단정이 중국 어선으로 접근하고 있다 사이렌을 울리며 내달린 해경 특수기동대원들은 어선에 올라 선장과 선원들을 통제하고 조업 규정 위반을 적발했다 성형주 기자 불법 조업해 숨겨 놓은 참조기 중국 어선 랴오좡위 선장오른쪽 등이 어창魚艙에 숨겼던 참조기를 꺼내자 해경 대원이 증거 사진을 찍는 모습 성형주 기자 불법 조업해 숨겨 놓은 참조기 중국 어선 랴오좡위 선장오른쪽 등이 어창魚艙에 숨겼던 참조기를 꺼내자 해경 대원이 증거 사진을 찍는 모습 성형주 기자 본지 이슬비 기자맨 오른쪽가 15일 중국 어선 랴오좡위호에서 어창魚艙을 내려다 보고 있다 서해해경청 3015함 본지 이슬비 기자맨 오른쪽가 15일 중국 어선 랴오좡위호에서 어창魚艙을 내려다 보고 있다 서해해경청 3015함 가거도 서쪽 40해리약 74㎞ 해상에서 조업 중인 중국 어선 40여척 발견 단속 실시 예정 특수기동대원 전원 집합15일 오전 11시쯤 서해지방해양경찰청 경비함인 3015함3000t급에 이틀째 되는 날이었다 뱃멀미 때문에 여경 선실 2층 침대 아래 칸에 누워 있었는데 긴급 방송이 나왔다 지난 6월 다시 독립조직이 해경은 요즘 무허가 조업 허가를 받았더라도 어획량을 지키지 않고 치어까지 쓸어가는 조업을 단속하고 있다 기자도 2박3일간의 일정으로 동행 취재에 나섰다방송을 듣고 조타실 갑판으로 올라가니 대원들이 검은색 진압복을 입고 모여 있었다 각자 방탄조끼에 K5 권총과 곤봉 섬광폭음탄 채증용 카메라를 장착한 상태였다 기자도 검은색

Article: 의무휴업제 시행 5년···대형마트·전통시장 둘다 침체된 까닭은?
서울뉴시스 최선윤 기자 대형마트 의무휴업제가 시행된 5년이 지난 가운데 당초 규제 취지와 달리 대형마트 전통시장 모두의 매출액이 감소하는 부정적인 효과가 나타나고 있다는 지적이 제기되면서 까닭은 무엇일 관심이 쏠린다27일 관련업계에 따르면 대형마트 의무휴업제가 도입된 배경은 전통시장 소상공인 보호에 있다 대형마트가 급격히 성장하자 해당 지역에서 상권을 공유하고 있는 중소유통업체와 소상공인들이 대형유통업체의 사업 확장에 대한 규제를 도입해야 한다는 주장을 지속했다이에 따라 지난 2005년부터 영업시간 영업품목 제한 대형유통업체의 사업 확장 규제를 내용으로 하는 다수의 의원입법안이 국회에 제출됐고 2012년 1월17일 유통산업발전법 일부개정법률안이 공포됨에 따라 지방자치단체는 조례를 통해 대규모점포 대형마트와 준대규모점포의 영업을 제한할 있게 됐다대형유통업체들로 구성된 한국체인스토어협회는 영업시간 제한과 의무 휴업일 강제가 소비자 불편과 소비 위축을 초래할 것이라며 규제를 반대했다 농수산물 매출 감소로 인한 농어민 피해 발생 일자리 축소 입점 자영업자와 중소협력업체의 매출 감소 등의 부작용의 발생도 피할 없을 것이라고 지적했다이같은 반발에도 의무휴업제는 시행됐고 실제 대형마트의 성장률은 하락했다 NH투자증권에 따르면 2012년 4월 대형마트 의무휴업이 최초 시행돼 2014년 12월까지 점포로 제도가 확대 적용되는 기간 동안 이마트와 롯데마트의 기존점 성장률은 평균 45 감소했다 영업이익 역시 인건비 임차료 고정비 부담으로 이마트는 22 롯데마트는 76 줄었다 때문에 이마트와 롯데쇼핑의 주가도 각각 하락 조정을 받았다하지만 문제는 전통시장 중소유통업체 소상공인들의 상황도 나아지지 않았다는 있다소상공인시장진흥공단에 따르면 전통시장의 평균 매출은 규제가 시작된 2012년 4755만원에서 2013년 4648만원으로 하락했다가 2014년 4672만원 2015년 4812만원으로 소폭 증가했다 물

Article: 지하철 9호선 파업 예고…지옥보다 더한 ‘지옥철’되나
서울 9호선운영노조 30일부터 6일간 한시적 파업 3조 2교대→5조 3교대 전환 1인역사 축소 휴식 보장 요구 퇴근시간대 평시대비 운행률 85수준으로 감소 우려 서울9호선운영 비상대책 마련운행차질 없다 서울 지하철 9호선 가양역 승강장이 지하철 이용객들로 붐비고 있다 사진연합뉴스 자료 서울지하철9호선 감사보고서 단위 억원 서울9호선운영노동조합은 27일 오전 서울 중구 민주노총에서 11월 30일 파업 돌입 기자회견을 하고 있다 사진 연합뉴스 이데일리 박철근 권오석 기자서울 지하철 9호선 1단계개화신논현역 운영을 담당하는 서울 9호선운영 노동조합은 27일 서울 중구 민주노총 사무실에서 기자회견을 열고 오는 30일부터 6일간 시한부 파업에 들어간다고 밝혔다 지난 2009년 회사 설립 이후 8년 만에 파업이다 노조는 시민의 안전과 생명을 지키기 위해서는 지하철 운행 유지 관리를 위한 인원 충원이 필요하다고 주장했다노조는 서울 지하철 18호선에 비해 현저하게 적은 인원으로 인해 노동강도가 과도하게 높다고 하소연한다노조 관계자는 18호선 지하철이 직원 1인당 16만명 내외를 수송하는 비해 9호선은 26만여명에 이른다 혼자 운전을 하는 기관사는 18호선 지하철 기관사보다 평균 23일을 일한다 25개역 가운데 15개역은 상시 1인 근무나 다름없는 상황 이라고 지적했다 이어 현재의 상황을 해소하기 위해서는 최소 45명에서 60명 가량 인력을 충원해야 한다 주장했다특히 노조는 회사가 수익을 승객과 직원 안전을위해 재투자하기보다는 외국 투자자 주머니를 채우기에 급급하다고 비난했다서울 지하철 18호선을 운영하는 서울교통공사가 연간 수천억원의 적자를 내고 있는 것에 비해 서울9호선운영은 흑자기조를 유지하고 있다 감사보고서에 따르면 서울9호선운영은 지난해 매출 675억원 영업이익 36억원 당기순이익 27억원을 기록했다노조에 따르면 지난 2009년 설립 이후 2015년까지 7년간 서울 9호선 운영에 투자한 트랑스데브코

Article: 판교 제2테크노밸리에 1200개 창업공간 만든다… "2022년 완공"
국토교통부 제공© News1 국토교통부 제공© News1 성남뉴스1 김희준 기자 빠르면 오는 2022년까지 임대료 부담없는 1200개 기업의 창업공간이 판교 제2테크노밸리판교 2밸리에 마련된다 1300석의 공유형 오피스와 1617가구의 주거공간도 조성된다정부는 11일 판교 2밸리 기업지원허브에서 열린 확대 경제관계장관회의를 통해 이같은 내용을 담은 판교 제2테크노밸리 활성화 방안을 발표했다이번 방안은 창업지원과 스타트업 업체 교류 지원프로그램으로 판교 2밸리를 창업 선도거점로 육성하기 위해 마련됐다활성화 방안에 따르면 먼저 창업자들이 임대료 부담없이 활용할 있는 창업자용 공간이 9개동 1200개사 규모로 늘어난다 기존 계획안에는 4개동 500개사 규모였다해당 공간은 LH한국토지주택공사가 마련한 기업성장센터 3동과 SW드림센터 성남시의 ICT융합센터에 조성된다여기에 선도 벤처기업이 연면적의 30를 창업기업 200개사에 무상 임대공간으로 제공하는 벤처타운도 마련된다사업성있는 창업 아이디어만 있으면 기술에서 금융컨설팅 해외진출까지 일괄지원하는 창업지원 프로그램도 판교 2밸리에 구축된다국토부 관계자는 정보화진흥원 공공기관들이 드론이나 자율주행차 헬스케어 11개 신산업 분야의 센터를 운영하며 창업을 지원하게 것이라고 말했다1인 창업자용 오픈카페와 스마트워크센터도 1300석 규모로 조성된다 공유형 오피스형식으로 누구나 이용할 있으며 특히 혁신카페엔 멘토링부스를 설치해 현직자가 예비창업자들에게 멘토링을 있도록 했다벤처타운에선 선도 벤처기업들이 후배기업들에게 컨설팅을 통한 사업노하우를 전수하는 시스템도 도입된다벤처타운 입주 스타트업 기업에게 함께 입주한 선도 벤처기업이 체계적인 지원프로그램을 제공하는 방식이다 선배 기업들이 공동으로 교육센터를 운영하며 혁신인력양성을 양성하는 프로그램도 진행된다국토부 관계자는 매출 1000억원이 넘는 중견 벤처기업들이 후배기업들을 선도하게 된다며 사회공여일 

Article: 유아까지 코딩 열풍…SW수업 의무화에 사교육시장 '꿈틀'
세종연합뉴스 고유선 기자 내년부터 학교에서 소프트웨어SW 교육이 필수화하면서 코딩컴퓨터 프로그래밍 교육과 관련된 사교육 시장이 들썩이고 있다5일 교육계에 따르면 대치동 반포동 목동을 비롯한 서울의 학원 밀집지역은 물론 분당 수도권에서도 코딩 학원이 속속 생겨나고 있다수업과 교과서의 가이드라인 역할을 하는 교육과정이 바뀌면서 초등학교는 2019년부터 5∼6학년 실과 과목에서 17시간 이상 중학교는 2018년부터 단계적으로 정보 과목에서 34시간 이상 SW 교육을 하게 된다대부분 학부모에게 이름마저 생소한 코딩교육이 이슈가 되면서 사교육 시장은 이미 바빠지고 있다올해부터 본격적으로 늘기 시작한 코딩 학원은 통상 일주일에 1∼2번 1개월∼6개월 과정을 만들고 코딩 중심의 소프트웨어 수업을 한다지역과 학생 연령별로 다르지만 달에 20만∼50만원을 받는다문제는 이들 학원이 초등학교에 입학한 아이들은 물론 미취학 아동에게도 코딩교육이 필요하다며 선행학습을 부추기는 데다 대학 진학에도 SW 교육이 필요하다고 광고하는 불안심리를 자극한다는 점이다강남구 학원은 핀란드의 경우 4세부터 코딩을 가르친다고 강조하며 홈페이지에서 5∼7세 유아를 가진 학부모들의 상담을 받고 있다서초구의 학원은 초등학교 입학과 동시에 코딩 교육을 시작해야 늦지 않는다고 강조하기도 했다학부모들은 창의력과 논리적 사고력을 높인다는 본연의 취지를 잃고 주입식 코딩교육이 자리를 잡게 될까 우려하는 모습이다초등학교 2학년 아들을 코딩 학원에 보내야 하는지 고민 중이라는 최현주37씨는 남편과 내가 모두 문과 출신이라 아이한테 미리 코딩 같은 것을 가르쳐줘야 한다는 압박감이 든다고 말했다그는 컴퓨팅적 사고력을 가르쳐준다느니 기존에 없던 신개념 수업이라느니 광고 전단지가 집에 많이 들어오는데 컴퓨팅적 사고력이 대체 뭔지 이해가 가지 않을뿐더러 결국은 교재를 달달 외우는 암기식 교육이 같아 고민이라고 말했다cindyynacokr
  Top-5 (